In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense

In [2]:
with open("sherlock-holm.es_stories_plain-text_advs.txt",'r',encoding='utf-8') as file:
    text = file.read()

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [4]:
total_words

8200

In [5]:
input_sequence = []

for line in text.split('\n'):
    toekn_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(toekn_list)):
        n_gram_sequence = toekn_list[:i+1]
        input_sequence.append(n_gram_sequence)

In [6]:
max_sequence_len = max([len(seq) for seq in input_sequence])
input_sequence = np.array(pad_sequences(input_sequence,maxlen=max_sequence_len,padding='pre'))

In [7]:
x = input_sequence[:, :-1]
y = input_sequence[:,-1]

In [8]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [9]:
model = Sequential()
model.add(Embedding(total_words, 100, input_shape=(max_sequence_len-1,)))  # Note the comma in (max_sequence_len-1,)
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

c:\Users\pc\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\core\embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 17, 100)        │       820,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 150)            │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 8200)           │     1,238,200 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,208,800 (8.43 MB)

 Trainable params: 2,208,800 (8.43 MB)

 Non-trainable params: 0 (0.00 B)

None


In [10]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x,y,epochs=100, verbose=1)

Epoch 1/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 66s 21ms/step - accuracy: 0.0621 - loss: 6.5445
Epoch 2/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 61s 20ms/step - accuracy: 0.1169 - loss: 5.5499
Epoch 3/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 60s 20ms/step - accuracy: 0.1458 - loss: 5.1371
Epoch 4/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 60s 20ms/step - accuracy: 0.1660 - loss: 4.7693
Epoch 5/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 60s 20ms/step - accuracy: 0.1852 - loss: 4.4334
Epoch 6/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 60s 20ms/step - accuracy: 0.2052 - loss: 4.1619
Epoch 7/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 64s 21ms/step - accuracy: 0.2366 - loss: 3.8660
Epoch 8/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 66s 22ms/step - accuracy: 0.2667 - loss: 3.6178
Epoch 9/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 66s 22ms/step - accuracy: 0.2995 - loss: 3.3713
Epoch 10/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 69s 23ms/step - accuracy: 0.3367 - loss: 3.1399
Epoch 11/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 65s 22ms/step - accuracy: 0.3743 - loss: 2.93

KeyboardInterrupt: 

In [ ]:
seed_text = input("Say Some Thing But Not Humble YourSelf: ")
next_word = 5

for _ in range(next_word):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1,padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)    

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
are you sure that everything is fastened '
